# Coursera assignment: Build a Regression Model in Keras

## D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

### Program Start

The predictors in the data of concrete strength include:

- Cement
- Blast Furnace Slag
- Fly Ash
- Water
- Superplasticizer
- Coarse Aggregate
- Fine Aggregate

Get the CSV file online

In [1]:
!wget -O concrete_data.csv https://cocl.us/concrete_data

--2020-04-09 04:36:10--  https://cocl.us/concrete_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 169.48.113.194, 158.85.108.83
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv [following]
--2020-04-09 04:36:11--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58988 (58K) [text/csv]
Saving to: ‘concrete_data.csv’

concrete_data.csv   100%[===================>]  57.61K  --.-KB/s    in 0.04s   

2020-04-09 04:36:11 

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_concrete_data = pd.read_csv('concrete_data.csv')
print("The shape is" , df_concrete_data.shape)
df_concrete_data.head()

The shape is (1030, 9)


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
df_concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
# always check the null data entry
df_concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The dataframe is good to go

## Split data into Predictor and Target

In [6]:
concrete_data_columns = df_concrete_data.columns

df_predictor = df_concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
df_target = df_concrete_data['Strength'] # Strength column

In [7]:
df_predictor.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
df_target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
print("The shape of the predictor is", df_predictor.shape)

n_cols = df_predictor.shape[1] # number of predictors
print("Input shape:", n_cols)

The shape of the predictor is (1030, 8)
Input shape: 8


## Normalize Data  **(B)

In [10]:
df_predictor_norm = (df_predictor - df_predictor.mean()) / df_predictor.std()
df_predictor_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [11]:
df_target_norm = (df_target - df_target.mean()) / df_target.std()
df_target_norm.head()

0    2.644123
1    1.560663
2    0.266498
3    0.313188
4    0.507732
Name: Strength, dtype: float64

## Build model using Keras

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [13]:
# define a regression model

def ann_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train Test Split (holding 30% of the data for testing)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_predictor_norm, df_target_norm, test_size=0.3, random_state=42)

print("# of test samples :", X_test.shape[0])
print("# of training samples:",X_train.shape[0])

# of test samples : 309
# of training samples: 721


## Train Model

In [15]:
# build the model
model = ann_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Train the model with 50 epochs (fit)
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
721/721 [==============================] - 1s 1ms/step - loss: 1.0114
Epoch 2/50
721/721 [==============================] - 0s 384us/step - loss: 0.8803
Epoch 3/50
721/721 [==============================] - 0s 391us/step - loss: 0.7866
Epoch 4/50
721/721 [==============================] - 0s 442us/step - loss: 0.7174
Epoch 5/50
721/721 [==============================] - 0s 412us/step - loss: 0.6618
Epoch 6/50
721/721 [==============================] - 0s 391us/step - loss: 0.6174
Epoch 7/50
721/721 [==============================] - 0s 444us/step - loss: 0.5816
Epoch 8/50
721/721 [==============================] - 0s 498us/step - loss: 0.5588
Epoch 9/50
721/721 [==============================] - 0s 469us/step - loss: 0.5362 0s - loss: 0.5
Epoch 10/50
721/721 [==============================] - 0s 446us/step - loss: 0.5180
Epoch 11/50
721/721 [==============================] - 0s 390us/step - loss: 0.5004
Epoch 12/50
721/721 [==============================] - 0s 442us/step - l

## Evaluate The Model

In [17]:
scores = model.evaluate(X_test, y_test)
# loss_val = model.evaluate(X_test, y_test, verbose=0) # to show the intermediate stage
y_pred = model.predict(X_test)
 
scores

309/309 [==============================] - 0s 329us/step


0.25276429285702195

## Compute the mean squared error

In [18]:
from sklearn.metrics import mean_squared_error

mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

0.2527643000930925 0.0


## Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [19]:
epochs = 50

total_mean_squared_errors = 50
mean_squared_errors = []

for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(df_predictor_norm, df_target_norm, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("Mean Squared Errors "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)


Mean Squared Errors 1: 0.15206258177371473
Mean Squared Errors 2: 0.14034269060517593
Mean Squared Errors 3: 0.101096046226503
Mean Squared Errors 4: 0.11302931237857318
Mean Squared Errors 5: 0.09756203328521507
Mean Squared Errors 6: 0.11841560027360144
Mean Squared Errors 7: 0.10646435420682901
Mean Squared Errors 8: 0.09090153123086324
Mean Squared Errors 9: 0.09204073882025808
Mean Squared Errors 10: 0.09777915665825594
Mean Squared Errors 11: 0.08523333781552546
Mean Squared Errors 12: 0.06803894542085315
Mean Squared Errors 13: 0.08532836551033564
Mean Squared Errors 14: 0.09061855699830842
Mean Squared Errors 15: 0.08254839265327238
Mean Squared Errors 16: 0.06991095513008945
Mean Squared Errors 17: 0.06553298620730157
Mean Squared Errors 18: 0.06880686813091383
Mean Squared Errors 19: 0.07083186128668029
Mean Squared Errors 20: 0.07680596678870395
Mean Squared Errors 21: 0.0746253033961293
Mean Squared Errors 22: 0.06907059512670757
Mean Squared Errors 23: 0.0682387424084361
M

## Report the mean and the standard deviation of the mean squared errors.

In [20]:
mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

Mean: 0.0746799806062013
Standard Deviation: 0.022110604427692193
